# Quantum State Preparation

Suppose you are given an arbitrary $n$-qubit state $|\psi\rangle = \sum_{x=0}^{2^n-1} c_x |x\rangle$, where $|x\rangle$ denotes the computational basis of $\mathbb{C}^{2^n},\ c_x\in\mathbb{C}\ \forall x$.  Our goal is to determine a quantum circuit $U$ which maps the state $|0\rangle^{\otimes n}$ onto $|\psi\rangle$.  Each $c_x$ can be written as $|c_x|e^{i\phi_x}$ for some angle $\phi_x$; thus, we can break the problem into two steps: 

 - **Initializing Amplitudes:** Create a gate which transforms the state $|0\rangle^{\otimes n}$ into the real-valued state $|\tilde\psi\rangle \equiv \sum_{x=0}^{2^n-1} |c_x| |x\rangle; |c_x|\in \mathbb{R}$.
 - **Initializing Phases:** Create a diagonal phase gate which transforms $|\tilde\psi\rangle$ into $|\psi\rangle$ by applying $\sum_{x=0}^{2^n-1} e^{i\phi_x} |x\rangle \langle x|$.

The algorithm used here for the amplitudes is due to Long and Sun [(arXiv)](https://arxiv.org/pdf/quant-ph/0104030) [(Phys. Rev. A)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.64.014303). It primarily relies on the (controlled-)gate 
$$U(\theta) \equiv \begin{pmatrix}\cos\theta & \sin\theta \\ \sin\theta & -\cos\theta \end{pmatrix}$$
which can be realized with `qiskit`'s `UGate(theta, phi=0, lambda=np.pi)`. We assume the **real** coefficients $c_x$ (dropping the absolute value signs momentarily) are labeled by the binary representation of the integers $0$ through $2^n-1$, denoted as $c_{x_n ... x_2 x_1 x_0}$, where $x_j \in \{0,1\}$ labels the state of the $j$<sup>th</sup> qubit $\left(j \in \{0,1,...,n-1\}\right)$.

1. For the $0$<sup>th</sup> qubit, we apply the gate $U(\theta_0)$, where
$$\theta_0 = 2\tan^{-1}\left(\frac{\sqrt{\sum_{x_1,x_2,...x_n} (c_{x_n ... x_2 x_1 1})^2}}{\sqrt{\sum_{x_1,x_2,...x_n} (c_{x_n ... x_2 x_1 0})^2}}\right)$$
and the sums run over all possible configurations of digits for $x_1$ through $x_n$. This rotates the state $|0\rangle^{\otimes n}$ into the state $
\cos(\theta_0)|0\rangle_0 \otimes |0\rangle^{\otimes n-1} + \sin(\theta_0)|1\rangle_0 \otimes |0\rangle^{\otimes n-1}$.

2. For the $k$<sup>th</sup> qubit $(k\geq 1)$, we denote by $|a\rangle$ the state of the previous $k$ qubits, where $a \equiv x_{k-1} ... x_1 x_0$ is the binary representation of an integer between $0$ and $2^k-1$. We loop over each possible configuration $a$ and apply a controlled-$U(\theta_{k,a})$ gate with
$$ \theta_{k,a}  = 2\tan^{-1} \left(\frac{\sqrt{\sum_{x_{k+1}, ... x_n} (c_{x_n ... x_{k+1},1,x_{k-1},... x_0})^2}}{\sqrt{\sum_{x_{k+1}, ... x_n} (c_{x_n ... x_{k+1},0,x_{k-1},... x_0})^2}}\right);$$
here the sums run over the configurations of qubits $k+1$ through $n$. The configuration of the first $k$ qubits is fixed by $a$, so this gate is applied only when qubits $0$ through $k-1$ are in the state $|a\rangle$. The multi-controlled $U(\theta_{k,a})$ rotates the state $|a\rangle \otimes |0\rangle^{\otimes (n-k)}$ into the state $|a\rangle \otimes \left(\cos\theta_{k,a} |0\rangle \otimes |0\rangle^{\otimes (n-k-1)} + \sin\theta_{k,a} |1\rangle \otimes |0\rangle^{\otimes (n-k-1)} \right)$.

Iterating over all the $n$ qubits requires $\sum_{j=0}^{n-1} 2^j$ gates of this type and transforms the state $|0\rangle^{\otimes n}$ into the state $\sum_x |c_x| |x\rangle$ (restoring the absolute values). To transform this into the final desired state, we need to apply multi-controlled phase gates which identify each state $|x\rangle$ then apply the global phase $e^{i\phi_x}$; this requires $2^n$ more gates and leaves us with the desired state 
$$|\psi\rangle = \sum_x |c_x| e^{i\phi_x} |x\rangle = \sum_x c_x |x\rangle.$$

Altogether, this algorithm requires a total of $\sum_{j=0}^n 2^j = 2^{n+1}-1$ operations acting on exactly $n$ qubits (no ancillas needed). As shown below, the algorithm can run in under 10 seconds for $n\leq 8$. 

<img src="amplitude_circuit.png" alt="Diagram of amplitude circuit, from Long and Sun" width="250"/>
Diagram of the $n=3$ circuit for the weights, from Long and Sun Fig. 1.

In [3]:
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import UGate
from qiskit.circuit.controlledgate import ControlledGate
import numpy as np
from numpy.linalg import norm
from itertools import combinations
import time

In [4]:
def zero_bits(s, q):
    '''
    For an integer `s`, 
        returns a list of which bits are 0 in the `q`-digit binary form
    '''
    return [i for i in range(q) if not (s >> i) & 1]

def one_bits(s, q):
    '''
    For an integer `s`, 
        returns a list of which bits are 1 in the `q`-digit binary form
    '''
    return [i for i in range(q) if (s >> i) & 1]

def find_digits(n0s, n1s, N):
    '''
    Given lists `n0s` and `n1s`, which contain binary positions of 0s and 1s,
        returns a list of the integers which contain those digits in their `N`-digit binary form;
        used to extract the correct subsets of coefficients from the state vector in defining θ_k,a
    '''
    return [ j for j in range(2**N) if all(((j >> pos) & 1) == 0 for pos in n0s) and all(((j >> pos) & 1) == 1 for pos in n1s) ]

def create_state_prep(state):
    '''
    Receives as input `state`, which must be a 2ⁿ-dimensional numpy array of complex values
    Outputs the circuit to prepare this state on a register of `n` qubits.
    '''
    N = len(state);
    n = int(np.log2(N));
    assert 2**n == N, "State must have length 2^n"
    state /= norm(state)       ## ensure normalization
    mags = np.abs(state);      ## extract real magnitudes of coefficients
    phases = np.angle(state);  ## extract arguments of coefficients

    state_qr = QuantumRegister(n, name = "s");
    prep_qc = QuantumCircuit(state_qr, name = "State Prep");

    ## Balancing the Weights ######################################################################################

    ## Qubit 0 - rotate |0> into cos(t)|0> + sin(t)|1>
    th = 2*np.arctan2(norm(mags[find_digits([], [0], n)]), norm(mags[find_digits([0], [], n)]) );
    prep_qc.u(th, 0, np.pi, state_qr[0]);
    prep_qc.barrier();

    if n>1:
        ## Qubits 1 through (n-1)
        for q in range(1,n):
            ## Loop over all configurations for qubits 0 through (q-1)
            for s in range(2**q):
                ## 
                xs = zero_bits(s,q);
                ys = one_bits(s,q);
                ## Flip bits corresponding to 0s in bitstring for `s`
                if xs: 
                    prep_qc.x(state_qr[xs]);
                ## Controlled U-gate to rotate |a>|0> into cos(t)|a>|0> + sin(t)|a>|1>
                th = 2*np.arctan2(norm(mags[find_digits(xs, ys+[q], n)]), norm(mags[find_digits(xs+[q], ys, n)]) );
                u = UGate(th, 0, np.pi).control(q);
                prep_qc.append(u, range(q+1));
                ## Unflip bits corresponding to 0s in bitstring for `s`
                if xs: 
                    prep_qc.x(state_qr[xs]);
    
            prep_qc.barrier();

    prep_qc.barrier();

    ## Diagonal Phase Unitary ######################################################################################

    for s in range(N):
        xs = zero_bits(s,n);
        ## Flip bits corresponding to 0s in bitstring for `s`
        if xs: 
            prep_qc.x(state_qr[xs]);
        ## Controlled phase to apply appropriate phase
        if n==1:
            prep_qc.p(phases[s], state_qr); 
        else:
            prep_qc.mcp(phases[s], state_qr[:-1], state_qr[-1]) 
        ## Unflip bits corresponding to 0s in bitstring for `s`
        if xs: 
            prep_qc.x(state_qr[xs]);

    ###############################################################################################################

    return prep_qc
    

In [5]:
## Input state as a 2ⁿ-dimensional numpy array
n = 3;

state = np.random.rand(2**n) + 1j * np.random.rand(2**n);
prep = create_state_prep(state);

s = Statevector(prep);
assert norm(s - state) < 1e-12; "ERROR: statevector does not match input state"

prep.draw()

┌───────────────┐ ░ ┌───┐                   ┌───┐                    ░ »
s_0: ┤ U(1.2862,0,π) ├─░─┤ X ├─────────■─────────┤ X ├─────────■──────────░─»
     └───────────────┘ ░ └───┘┌────────┴────────┐└───┘┌────────┴────────┐ ░ »
s_1: ──────────────────░──────┤ U(1.6532,0,π,0) ├─────┤ U(2.1093,0,π,0) ├─░─»
                       ░      └─────────────────┘     └─────────────────┘ ░ »
s_2: ──────────────────░──────────────────────────────────────────────────░─»
                       ░                                                  ░ »
«     ┌───┐                 ┌───┐                       ┌───┐                 »
«s_0: ┤ X ├────────■────────┤ X ├─────────────■─────────┤ X ├────────■────────»
«     ├───┤        │        ├───┤┌───┐        │         ├───┤        │        »
«s_1: ┤ X ├────────■────────┤ X ├┤ X ├────────■─────────┤ X ├────────■────────»
«     └───┘┌───────┴───────┐└───┘└───┘┌───────┴────────┐└───┘┌───────┴───────┐»
«s_2: ─────┤ U(1.9762,0,π) ├──────────┤ U(0.79367,0,π) ├─────┤ U(1.3832,0,π) ├»
«          └───────────────┘          └────────────────┘     └───────────────┘»
«     ┌───┐                  ░  ░ ┌───┐            ┌───┐                 ┌───┐»
«s_0: ┤ X ├────────■─────────░──░─┤ X ├─■──────────┤ X ├──────■──────────┤ X ├»
«     └───┘        │         ░  ░ ├───┤ │          ├───┤┌───┐ │          ├───┤»
«s_1: ─────────────■─────────░──░─┤ X ├─■──────────┤ X ├┤ X ├─■──────────┤ X ├»
«          ┌───────┴───────┐ ░  ░ ├───┤ │P(1.2986) ├───┤├───┤ │P(1.2097) ├───┤»
«s_2: ─────┤ U(1.7861,0,π) ├─░──░─┤ X ├─■──────────┤ X ├┤ X ├─■──────────┤ X ├»
«          └───────────────┘ ░  ░ └───┘            └───┘└───┘            └───┘»
«                      ┌───┐                  ┌───┐            ┌───┐     »
«s_0: ──────■──────────┤ X ├──────■───────────┤ X ├─■──────────┤ X ├─────»
«           │          └───┘      │           ├───┤ │          ├───┤┌───┐»
«s_1: ──────■─────────────────────■───────────┤ X ├─■──────────┤ X ├┤ X ├»
«     ┌───┐ │P(1.5245) ┌───┐┌───┐ │P(0.62579) ├───┤ │P(1.3463) └───┘└───┘»
«s_2: ┤ X ├─■──────────┤ X ├┤ X ├─■───────────┤ X ├─■────────────────────»
«     └───┘            └───┘└───┘             └───┘                      »
«                 ┌───┐            ┌───┐             
«s_0: ─■──────────┤ X ├─■──────────┤ X ├─■───────────
«      │          ├───┤ │          └───┘ │           
«s_1: ─■──────────┤ X ├─■────────────────■───────────
«      │P(1.0989) └───┘ │P(0.9896)       │P(0.97067) 
«s_2: ─■────────────────■────────────────■───────────
«

In [6]:
## Input state as a 2ⁿ-dimensional numpy array
n = 8;

state = np.random.rand(2**n) + 1j * np.random.rand(2**n);

In [7]:
start = time.time()

prep = create_state_prep(state);
s = Statevector(prep);
assert norm(s - state) < 1e-12; "ERROR: statevector does not match input state"

print("Runs in ", np.round(time.time() - start, decimals=3), " seconds")

Runs in  7.297  seconds


## Acknowledgements

**Magnitude Algorithm:**
Long, G. L., & Sun, Y. (2001). "Efficient scheme for initializing a quantum register with an arbitrary superposed state." [*Physical Review A, 64(1), 014303*](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.64.014303).

Some of the Python syntax/code used here was developed with the aid of Microsoft Copilot, as well as heavy reference to the [Qiskit Documentation](https://docs.quantum.ibm.com).